In [1]:
from mne.io import read_raw_edf
import numpy as np
import pandas as pd

03 [2996 3036]
04 [1467 1494]
15 [1732 1772]
16 [1015 1066]
18 [1720 1810]
21 [327 420]
26 [1862 1963]

In [2]:
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_03' + '.edf', preload=False, verbose='ERROR')
ch_names = file.ch_names
fs = int(file.info['sfreq'])

In [3]:
hour3 = (np.arange(2996, 3037, step=1/fs) * int(fs)).astype(int)
hour4 = (np.arange(1467, 1495, step=1/fs) * int(fs)).astype(int)
hour15 = (np.arange(1732, 1773, step=1/fs) * int(fs)).astype(int)
hour16 = (np.arange(1015, 1067, step=1/fs) * int(fs)).astype(int)
hour18 = (np.arange(1720, 1811, step=1/fs) * int(fs)).astype(int)
hour21 = (np.arange(327, 421, step=1/fs) * int(fs)).astype(int)
hour26 = (np.arange(1862, 1964, step=1/fs) * int(fs)).astype(int)

In [4]:
num_true = len(hour3) + len(hour4) + len(hour15) + len(hour16) + len(hour18) + len(hour21) + len(hour26)
num_true

114944

In [5]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_03' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (0 – Normal, 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 2996 <= (i / fs) < 3037 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :])
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index)

# Undersample Class 0 samples via random selection to create balanced class ratio
x_temp2 = pd.concat([x_temp2, x_temp1.sample(n=len(x_temp2), replace=False, axis='index')]).reset_index(drop=True)

x_data = x_temp2.copy()
print(x_data.shape)

(20992, 24)


In [6]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_04' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1467 <= (i / fs) < 1495 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
x_temp2 = pd.concat([x_temp2, x_temp1.sample(n=len(x_temp2), replace=False, axis='index')]).reset_index(drop=True)

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(35328, 24)


In [7]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_15' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1732 <= (i / fs) < 1773 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
x_temp2 = pd.concat([x_temp2, x_temp1.sample(n=len(x_temp2), replace=False, axis='index')]).reset_index(drop=True)

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(56320, 24)


In [8]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_16' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1015 <= (i / fs) < 1067 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
x_temp2 = pd.concat([x_temp2, x_temp1.sample(n=len(x_temp2), replace=False, axis='index')]).reset_index(drop=True)

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(82944, 24)


In [9]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_18' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1720 <= (i / fs) < 1811 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
x_temp2 = pd.concat([x_temp2, x_temp1.sample(n=len(x_temp2), replace=False, axis='index')]).reset_index(drop=True)

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(129536, 24)


In [10]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_21' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 327 <= (i / fs) < 421 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
x_temp2 = pd.concat([x_temp2, x_temp1.sample(n=len(x_temp2), replace=False, axis='index')]).reset_index(drop=True)

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(177664, 24)


In [11]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_26' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1862 <= (i / fs) < 1964 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
x_temp2 = pd.concat([x_temp2, x_temp1.sample(n=len(x_temp2), replace=False, axis='index')]).reset_index(drop=True)

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(229888, 24)


In [12]:
x_data.iloc[:, 1:] = x_data.iloc[:, 1:].multiply(1E6)
x_data.head()

,State,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,...,F8-T8,T8-P8-0,P8-O2,FZ-CZ,CZ-PZ,P7-T7,T7-FT9,FT9-FT10,FT10-T8,T8-P8-1
0,1,57.240537,45.909646,55.677656,3.321123,-21.684982,42.393162,84.590965,57.631258,10.354090,...,45.518926,32.625153,-57.240537,42.783883,113.894994,-55.286935,18.949939,24.810745,6.446886,32.625153
1,1,53.333333,50.989011,56.459096,2.539683,-19.731380,45.518926,88.888889,50.207570,25.592186,...,39.267399,21.294261,-51.379731,35.750916,114.676435,-56.068376,15.824176,36.923077,9.181929,21.294261
2,1,45.909646,59.194139,54.896215,-0.976801,-23.247863,50.207570,88.888889,43.174603,38.876679,...,46.300366,10.744811,-70.134310,25.592186,113.504274,-54.505495,3.711844,54.896215,12.698413,10.744811
3,1,42.393162,64.273504,52.161172,-4.102564,-22.075702,54.505495,83.028083,40.048840,45.518926,...,50.989011,3.711844,-81.855922,20.122100,113.894994,-51.770452,-6.446886,68.962149,15.824176,3.711844
4,1,46.300366,58.021978,46.691087,-2.539683,-16.214896,53.333333,80.293040,31.062271,50.989011,...,40.830281,-5.665446,-76.776557,21.684982,115.457875,-46.300366,2.930403,73.650794,12.698413,-5.665446


In [13]:
x_data.to_csv(path_or_buf='data.csv', index=False)